In [1]:
import pandas as pd
import math
import csv

In [2]:
hh_cols = ['hhlivingsituation', 'hhnumdependentchildren', 'hhvehiclenumber', 'familyincomelevel', 'HmTTS2006']
hh_rename = ['Liv_Arr', 'Children', 'Cars', 'Income', 'Home_Zone']
df = pd.read_csv('SMTO_2019/SMTO_2019_Households.csv')[hh_cols]
df = df.rename(columns = dict(zip(hh_cols, hh_rename)))
df.head()

,Liv_Arr,Children,Cars,Income,Home_Zone
0,Live with family/parents,0.0,0.0,NaN,3851.0
1,Live with family/parents,0.0,1.0,NaN,181.0
2,Live with family/parents,0.0,2.0,I don't know,1039.0
3,NaN,NaN,NaN,NaN,191.0
4,Live with family/parents,1.0,1.0,NaN,600.0


In [3]:
ps_cols = ['psinstitution', 'personstatusgrad ', 'personstatustime', 'pscollegeaffiliation', 'pscampusmain',
          'psworknumhoursperweek', 'psdrivinglicenseowner', 'psmainmodefalltypical']
ps_rename = ['School', 'Uni_Level', 'Uni_Status', 'Col_Status', 'Campus', 'Work', 'Licence', 'Mode']
df = df.join(pd.read_csv('SMTO_2019/SMTO_2019_Respondents.csv')[ps_cols])
df = df.rename(columns = dict(zip(ps_cols, ps_rename)))
df.head()

C:\Users\ethan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Liv_Arr,Children,Cars,Income,Home_Zone,School,Uni_Level,Uni_Status,Col_Status,Campus,Work,Licence,Mode
0,Live with family/parents,0.0,0.0,NaN,3851.0,Centennial College,NaN,NaN,Full-Time,Progress Campus,I don't work,No,GO Bus
1,Live with family/parents,0.0,1.0,NaN,181.0,Centennial College,NaN,NaN,Part-Time,Morningside Campus,I don't work,No,Transit Bus
2,Live with family/parents,0.0,2.0,I don't know,1039.0,Centennial College,NaN,NaN,Full-Time,Progress Campus,I don't work,No,GO Bus
3,NaN,NaN,NaN,NaN,191.0,Centennial College,NaN,NaN,Full-Time,Progress Campus,I don't work,No,NaN
4,Live with family/parents,1.0,1.0,NaN,600.0,Centennial College,NaN,NaN,Full-Time,Progress Campus,I don't work,No,NaN


In [4]:
df['Campus'] = df.apply(lambda x: x.School if x.School in ('Ryerson University', 'OCAD University') else x.Campus, axis=1)
df['School_Type'] = df['School'].apply(lambda x: 'College' if 'College' in x else 'University')
df['Col_Status'].replace({'Full-Time': 'FT', 'Part-Time ':'PT', 'Continuing Education': 'Other'}, inplace=True)
df['Work'].replace({"I don't work": 'NW', 'I work 11-20 hours per week': 'PT', 'I work < 11 hours per week': 'PT', 'I work 21-30 hours per week': 'PT', 'I work 31-40 hours per week': 'FT', 'I work > 40 hours per week': 'FT'}, inplace=True)
df['Licence'] = df['Licence'].apply(lambda x: True if x=="Yes" else (False if x=="No" else None))
df['Family'] = df['Liv_Arr'].apply(lambda x: (x == 'Live with family/parents') if isinstance(x, str) else None)

In [5]:
def row_to_level(x):
    return x.Uni_Level if isinstance(x.Uni_Level, str) else ("College" if x.School_Type == 'College' else None)
def row_to_status(x):
    return x.Uni_Status if isinstance(x.Uni_Status, str) else (x.Col_Status if isinstance(x.Col_Status, str) else None)

df['Level'] = df.apply(row_to_level, axis=1)
df['Status'] = df.apply(row_to_status, axis=1)
df.drop(columns=['Uni_Level', 'Uni_Status', 'Col_Status'], inplace=True)

In [6]:
active = ['Walk', 'Bicycle', 'Bikeshare']
auto = ['Drive alone', 'Auto passenger (driver is a household member)', 'Drive with passenger(s) (household members only)',
        'Accessibility adapted vehicle as driver', 'Drive with passenger(s) (including non-household members)', 'Accessibility adapted vehicle as passenger',
       'Ride-hailing alone (UberX, Lyft etc.)', 'Auto passenger (driver is a non-household member)', 'Taxi', 'Ride-hailing with other passengers (Uberpool, Lyftpool etc.)']
transit = ['Transit Bus', 'Subway/RT', 'GO Train', 'GO Bus', 'Paratransit service (e.g., bus transport for disabled persons)', 'Streetcar']
def mode_to_aggr(mode):
    if mode in active:
        return 'Active'
    elif mode in auto:
        return 'Auto'
    elif mode in transit:
        return 'Transit'
    elif mode == 'I do not travel to the university (distance learners only)':
        return 'Do not travel'
    elif isinstance(mode, str):
        return 'Other'
    else:
        return None
    
df['Mode'] = df['Mode'].apply(mode_to_aggr)

In [7]:
df = df[(df['Status'] == 'FT') | (df['Status'] == 'PT')]
df = df[df['Campus'].notnull()]
df = df[df['Mode'] != 'Do not travel']
df = df[df['Home_Zone'].notnull()]

Drop Status=Other, Campus=Unknown?, Do not travel, missing HZ
Replace unknowns and np.nans
Reformat income

In [8]:
campus_info = pd.read_csv('School_Info_2019.csv').set_index('Campus')
zones, codes = list(campus_info['Zone']), list(campus_info['Code'])
df['Campus_Zone'] = df['Campus'].apply(lambda x: campus_info['Zone'].loc[x] if isinstance(x, str) else -1)
campus_info.head()

,Code,Zone,UG,Grad,Total,Tuition,Domestic_UG,Domestic_Grad,Admission
Campus,,,,,,,,,
Story Arts Centre,CST,282,NaN,NaN,"1,425",NaN,NaN,NaN,NaN
Ashtonbee Campus,CAS,526,NaN,NaN,"2,332",NaN,NaN,NaN,NaN
Progress Campus,CPR,493,NaN,NaN,"11,871",NaN,NaN,NaN,NaN
Morningside Campus,CMO,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Downsview Campus,CDV,419,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_path = pd.read_csv('../../LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

In [10]:
def get_distance(o, d):
    if o in origins and d in origins:
        i = origins.index(o)
        j = origins.index(d)
        return dists[i*2392 + j] / 1000
    else:
        return -1    

In [11]:
for i in range(len(codes)):
    df['Dist.' + codes[i]] = df['Home_Zone'].apply(lambda x: get_distance(x, zones[i]))
df = df[df['Dist.CST'] != -1]

Load campus data from campus_info
Take correct enrollment and domestic values by level/status
Decide how to handle expansion factors

In [ ]:
grouped = df.groupby('Level')['School'].value_counts()
school_counts = df['School'].value_counts()

def level_count(school, level):
    if "College" in school:
        return grouped['College'][school] / grouped.sum()
    if level != 'Total':
        return grouped[level][school] / grouped.sum()
    return school_counts[school]

In [ ]:
def get_log_enrollment(level, school):
    return math.log1p(campus_info[level].loc[school]) # Level: UG, Grad, Total
    
exp_factors = {'Total': {}, 'UG': {}, 'Grad': {}}
for school in campus_info.index:
    code = campus_info['Code'][school]
    for level in ('Total', 'UG', 'Grad'):
        exp_factors[level][school] = campus_info[level][school] / campus_info[level].sum() / level_count(school, level)
        df[level + "." + code] = get_log_enrollment(level, school)
    df["Tuition." + code] = campus_info['Tuition'][school]
    df["Domestic." + code] = df['Level'].apply(lambda x: campus_info['Domestic_' + ('Grad' if (x == 'Grad') else 'UG')][school])  
    df["Adm_Ave." + code] = campus_info['Admission'][school]
    
    exp_factors[school] = campus_info[]

In [13]:
df.to_csv('SMTO_2019/SMTO_2019_Complete_Input.csv', index = False)